# このCodeの目的
- 2次元スライス画像を、3次元のnp.array配列に変換する
- 高さ×幅の断面だけでなく、深度方向の断面図を可視化する
- case, day を指定することで、任意の断面図を可視化できるようにする

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image
import os, shutil
import math

# 1. 画像をそのまま表示（深さ方向で垂直にスライスした断面図）

In [ ]:
def display_xy_imgs(CASE, DAY, slice_list):
    # subplot用の高さ設定
    plot_line = math.ceil(len(slice_list)/5)
    
    # train画像のpath取得
    TRAIN_DIR='../input/uw-madison-gi-tract-image-segmentation/train/case'+CASE+'/case'+CASE+'_day'+DAY+'/scans/'
    train_images = glob(os.path.join(TRAIN_DIR, '**', '*.png'), recursive=True)
    train_images = sorted(train_images)
    
    # 画像表示
    fig = plt.figure(figsize=(20,plot_line*4))
    i=1
    for slice_i in slice_list:
        ax = fig.add_subplot(plot_line,5,i)
        ax.set_title('slice ' + str(slice_i))
        im = Image.open(train_images[slice_i-1])
        img = ax.imshow(np.asarray(im),cmap=plt.cm.jet)
        cbar = fig.colorbar(img, ax=ax, aspect=50, pad=0.08, shrink=0.9, orientation='vertical')
        i+=1
    plt.suptitle('case' + CASE + ' day' + DAY,fontsize=16)
    plt.show()

In [ ]:
# 見たい画像の指定
CASE = '2'
DAY = '1'
slice_list = range(1,144,10) #スライス番号 1始まり

display_xy_imgs(CASE, DAY, slice_list)

# 2. 3次元のnp.array配列作成

In [ ]:
def make_3d_img_array(CASE, DAY):
    # train画像のpath取得
    TRAIN_DIR='../input/uw-madison-gi-tract-image-segmentation/train/case'+CASE+'/case'+CASE+'_day'+DAY+'/scans/'
    train_images = glob(os.path.join(TRAIN_DIR, '**', '*.png'), recursive=True)
    train_images = sorted(train_images)

    # 画像のサイズ取得
    im = np.asarray(Image.open(train_images[0]))
    height = len(im)
    width = len(im[0])

    # 2次元画像を3次元配列に格納
    im_3d = np.empty((0, height, width))

    for path in train_images:
        im = np.asarray(Image.open(path))
        im = np.reshape(im, (1, height, width))
        im_3d = np.append(im_3d, im, axis = 0)

    print('size:{}'.format(im_3d.shape))
    return im_3d

In [ ]:
# 確認したい画像の指定
CASE = '2'
DAY = '1'

# 3次元配列作成
im_3d = make_3d_img_array(CASE, DAY)

# 3. 深さ、高さ、幅の中から任意の軸でスライスした断面図を表示

In [ ]:
def display_slice_3d_img(im_3d, slice_list, axis):
    # subplot用の高さ設定
    plot_line = math.ceil(len(slice_list)/5)
    axis_name = ['depth','height','width']
    
    # 画像表示
    fig = plt.figure(figsize=(20,plot_line*4))
    i=1
    for slice_i in slice_list:
        ax = fig.add_subplot(plot_line,5,i)
        ax.set_title('slice ' + str(slice_i))
        if axis == 0:
            img = ax.imshow(im_3d[slice_i,:,:],cmap=plt.cm.jet)
        elif axis == 1:
            img = ax.imshow(im_3d[:,slice_i,:],cmap=plt.cm.jet)
        elif axis == 2:
            img = ax.imshow(im_3d[:,:,slice_i],cmap=plt.cm.jet)
        cbar = fig.colorbar(img, ax=ax, aspect=50, pad=0.08, shrink=0.9, orientation='vertical')
        i+=1
    plt.suptitle('Slice perpendicular to  {}'.format(axis_name[axis]) ,fontsize=16)
    plt.show()

## 深さ方向に垂直にスライスした断面図（画像そのままと同じ）

In [ ]:
axis = 0 # 0:depth, 1:height, 2:width
slice_list = range(0,len(im_3d),10) #スライス番号 0始まり

# 断面図の表示
display_slice_3d_img(im_3d, slice_list, axis)

## 高さ方向に垂直にスライスした断面図

In [ ]:
axis = 1 # 0:depth, 1:height, 2:width
slice_list = range(0,len(im_3d[0]),10) #スライス番号 0始まり

# 断面図の表示
display_slice_3d_img(im_3d, slice_list, axis)

## 幅方向に垂直にスライスした断面図

In [ ]:
axis = 2 # 0:depth, 1:height, 2:width
slice_list = range(0,len(im_3d[0][0]),10) #スライス番号 0始まり

# 断面図の表示
display_slice_3d_img(im_3d, slice_list, axis)

高さ方向に垂直にスライスした断面図、幅方向に垂直にスライスした断面図を見ると、若干滑らかに画像が繋がっていないことが分かる。<br>
中心付近には背骨が見えたり、素人目でもいくつかの情報は読み取れる。

# 4. 輝度値を確認して前処理による表示改善を試す（結論上手くいかなかった）

3次元画像の輝度値をヒストグラム表示。<br>

In [ ]:
brightness = im_3d.ravel()
plt.hist(brightness, bins=100)
plt.xlabel('brightness')
plt.show()

max_brightness = brightness.max()
print('max brightness: {}'.format(max_brightness))

通常、輝度値は0-255の値で表されるが、255を超えている輝度値が存在する。<br>
そこで、単純な前処理で0-255の値に変換し、表示が改善されるか試してみる。<br>
4-1. クリップ処理：255を超える値を255に切り下げ<br>
4-2. スケール処理：255を超える場合、最大輝度値で割ってスケーリング

## 4-1. クリップ処理

In [ ]:
# 最小0、最大255でクリップ
im_3d_clipped = np.clip(im_3d,0,255)

# ヒストグラム確認
brightness = im_3d_clipped.ravel()
plt.hist(brightness, bins=100)
plt.xlabel('brightness')
plt.show()

max_brightness = brightness.max()
print('max brightness: {}'.format(max_brightness))

## （クリップ処理）高さ方向に垂直にスライスした断面図

In [ ]:
axis = 1 # 0:depth, 1:height, 2:width
slice_list = range(0,len(im_3d[0]),10) #スライス番号 0始まり

# 断面図の表示
display_slice_3d_img(im_3d_clipped, slice_list, axis)

## （クリップ処理）幅方向に垂直にスライスした断面図

In [ ]:
axis = 2 # 0:depth, 1:height, 2:width
slice_list = range(0,len(im_3d[0]),10) #スライス番号 0始まり

# 断面図の表示
display_slice_3d_img(im_3d_clipped, slice_list, axis)

クリップ処理では上手く描画することはできなかった。単純に情報量落としているだけなので、当然といえば当然。

## 4-2. スケール処理

In [ ]:
# 元のスライス画像ごとに、255を超える輝度値が存在すればスケーリング
im_3d_scaled = im_3d.copy()
for i in range(len(im_3d)):
    if im_3d[i].max() > 255:
        im_3d_scaled[i] = (im_3d[i] / im_3d[i].max())* 255

# ヒストグラム確認
brightness = im_3d_scaled.ravel()
plt.hist(brightness, bins=100)
plt.xlabel('brightness')
plt.show()

max_brightness = brightness.max()
print('max brightness: {}'.format(max_brightness))

## （スケール処理）高さ方向に垂直にスライスした断面図

In [ ]:
axis = 1 # 0:depth, 1:height, 2:width
slice_list = range(0,len(im_3d[0]),10) #スライス番号 0始まり

# 断面図の表示
display_slice_3d_img(im_3d_scaled, slice_list, axis)

## （スケール処理）幅方向に垂直にスライスした断面図

In [ ]:
axis = 2 # 0:depth, 1:height, 2:width
slice_list = range(0,len(im_3d[0]),10) #スライス番号 0始まり

# 断面図の表示
display_slice_3d_img(im_3d_scaled, slice_list, axis)

こちらも綺麗に描画することはできなかった。<br>
無理やり肯定的に捉えると、輝度値のもともと高かった体表付近は少し綺麗になったと言えるかもしれないが、肝心の内部構造は明らかに元画像より崩れている。単純な前処理で解決する問題ではなさそうだ。<br>
次は、この画素値がどのような確率分布に従っているかを分析したい。